In [35]:
# -*- coding: utf-8 -*-
"""
SCRIPT OTIMIZADO: EXTRAÇÃO EM LOTE POR PIXEL DE IMAGENS EMIT
==================================================================
Versão 2.0 - Refatorado para eficiência e resiliência

• Implementa processamento em lote (batch) para minimizar requisições ao GEE.
• Move a lógica de extração para o lado do servidor com .map().
• Salva os resultados incrementalmente a cada lote processado.
• Adiciona uma etapa de diagnóstico para inspecionar as propriedades do Asset.

Autor: Sistema Otimizado de Sensoriamento Remoto (Refatorado por Gemini)
Data: Agosto 2025
"""

import ee
import pandas as pd
import os
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Tenta importar do Colab, se não, ignora (para uso local)
try:
    from google.colab import drive
except ImportError:
    drive = None

print("🚀 SCRIPT OTIMIZADO: EXTRAÇÃO EM LOTE POR PIXEL")
print("="*70)

# =============================================================================
# 1. CONFIGURAÇÕES
# =============================================================================

class OptimizedConfig:
    """Configurações centralizadas para o script otimizado."""
    def __init__(self):
        # Arquivos e pastas
        self.paths = {
            'input_sheet_asset_id': 'projects/ee-vladimircambiental/assets/pontos_amostrados',
            'study_area_asset_id': 'projects/ee-crypto/assets/Vegetacao_CE_Shapefile',
            'output_folder': '/content/drive/My Drive/GEE_Exports_Pixel_Batch',
            'output_prefix': 'emit_batch_extracted'
        }
        # Configurações temporais
        self.temporal_config = {
            'start_date': '2022-08-01',
            'end_date': datetime.now().strftime('%Y-%m-%d'),
        }
        # Máscara de qualidade
        self.quality_masks = {
            'aggregate_flag_band': 'aggregate_flag',
            'cloud_bit': 10,
            'aerosol_bit': 7,
        }
        # Propriedades a serem mantidas do Asset original
        self.output_properties = ['UniFitoMor', 'mapbiomas_class']
        # Configuração do processamento em lote
        self.batch_config = {
            'batch_size': 15  # Número de imagens a processar por requisição ao GEE
        }
        # Configuração de seleção de classes
        self.selection_config = {
            'enabled': True,
            # ⚠️ ATENÇÃO: Verifique os nomes exatos no diagnóstico antes de rodar!
            'selected_classes': ['Formação Florestal', 'Formação Savânica', 'Mangue', 'Restinga Arborizada'],
            'filter_by_property': 'mapbiomas_class'
        }

config = OptimizedConfig()

# =============================================================================
# 2. FUNÇÕES AUXILIARES E DE DIAGNÓSTICO
# =============================================================================

def initialize_gee():
    """Inicializa e autentica o Google Earth Engine."""
    try:
        ee.Initialize(project='ee-vladimircambiental')
    except Exception as e:
        print("Autenticando GEE...")
        ee.Authenticate()
        ee.Initialize(project='ee-vladimircambiental')
    print("✅ Google Earth Engine inicializado")

def mount_drive_colab():
    """Monta o Google Drive se estiver no ambiente Colab."""
    if drive:
        drive.mount('/content/drive', force_remount=True)
        print("✅ Google Drive montado")

def inspect_asset_properties(asset_id, num_features=5):
    """Carrega algumas feições de um Asset e exibe suas propriedades para diagnóstico."""
    print(f"\n🕵️ INSPECIONANDO PROPRIEDADES DO ASSET: {asset_id}")
    try:
        fc = ee.FeatureCollection(asset_id).limit(num_features)
        features = fc.getInfo()['features']
        if not features:
            print("❌ O Asset parece estar vazio ou inacessível.")
            return
        print(f"   Propriedades encontradas na primeira feição: {list(features[0]['properties'].keys())}")
        print(f"   Exemplos de valores para '{config.selection_config['filter_by_property']}':")
        for f in features:
            prop_value = f['properties'].get(config.selection_config['filter_by_property'], 'NÃO ENCONTRADO')
            print(f"     - '{prop_value}'")
    except Exception as e:
        print(f"❌ Erro ao inspecionar o Asset: {e}")

# =============================================================================
# 3. FUNÇÕES PRINCIPAIS (LÓGICA GEE SERVER-SIDE)
# =============================================================================

def apply_pixel_cloud_mask(image):
    """Função Server-Side para aplicar máscara de nuvem."""
    flag_band = image.select(config.quality_masks['aggregate_flag_band'])
    qa_int = flag_band.cast({'aggregate_flag': 'int'})
    cloud_mask = qa_int.bitwiseAnd(1 << config.quality_masks['cloud_bit']).eq(0)
    aerosol_mask = qa_int.bitwiseAnd(1 << config.quality_masks['aerosol_bit']).eq(0)
    clean_mask = cloud_mask.And(aerosol_mask)
    return image.updateMask(clean_mask)

def extract_data_from_image(image, points_fc):
    """
    Função Server-Side que processa UMA imagem: mascara, seleciona bandas
    e extrai dados para a coleção de pontos.
    """
    image = ee.Image(image) # Cast para garantir que é um ee.Image
    image_id = image.id()
    image_timestamp = image.get('system:time_start')

    image_masked = apply_pixel_cloud_mask(image)

    # Extrai valores para os pontos desta imagem
    sampled_features = image_masked.sampleRegions(
        collection=points_fc,
        scale=60,
        geometries=True,
        properties=config.output_properties
    )

    # Adiciona metadados da imagem a cada ponto extraído
    def add_metadata(feature):
        return feature.set({
            'image_id': image_id,
            'image_timestamp': image_timestamp
        })

    # Retorna uma FeatureCollection com dados e metadados, ou vazia se nada foi extraído
    return sampled_features.map(add_metadata)

# =============================================================================
# 4. PIPELINE PRINCIPAL DE PROCESSAMENTO EM LOTE
# =============================================================================

def main_batch_pipeline(config):
    """Pipeline otimizado que processa imagens em lotes e salva incrementalmente."""
    start_time = time.time()
    print("\n🚀 EXECUTANDO PIPELINE DE EXTRAÇÃO EM LOTE")
    print("="*60)

    # 1. Inicialização e configuração de pastas
    initialize_gee()
    mount_drive_colab()
    os.makedirs(config.paths['output_folder'], exist_ok=True)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_path = os.path.join(
        config.paths['output_folder'],
        f"{config.paths['output_prefix']}_{timestamp}.csv"
    )
    print(f"💾 Arquivo de saída será salvo incrementalmente em: {output_path}")


    # 2. Diagnóstico e carregamento dos pontos
    inspect_asset_properties(config.paths['input_sheet_asset_id'])
    points_fc = ee.FeatureCollection(config.paths['input_sheet_asset_id'])

    if config.selection_config['enabled']:
        print(f"\n🎯 FILTRANDO PONTOS pela propriedade '{config.selection_config['filter_by_property']}'...")
        class_filter = ee.Filter.inList(
            config.selection_config['filter_by_property'],
            config.selection_config['selected_classes']
        )
        points_fc = points_fc.filter(class_filter)
        total_points = points_fc.size().getInfo()
        print(f"✅ Total de pontos após filtro: {total_points:,}")
        if total_points == 0:
            print("❌ O filtro resultou em 0 pontos. Verifique os nomes em 'selected_classes' com base na inspeção acima. Abortando.")
            return

    # 3. Carregamento da área de estudo e da coleção de imagens
    study_area = ee.FeatureCollection(config.paths['study_area_asset_id']).geometry().dissolve()
    emit_collection = (ee.ImageCollection('NASA/EMIT/L2A/RFL')
                       .filterDate(config.temporal_config['start_date'], config.temporal_config['end_date'])
                       .filterBounds(study_area))

    image_ids = emit_collection.aggregate_array('system:id').getInfo()
    total_images = len(image_ids)
    if total_images == 0:
        print("❌ Nenhuma imagem EMIT encontrada para a área e período. Abortando.")
        return
    print(f"\n🛰️ {total_images} imagens EMIT encontradas. Iniciando processamento em lotes de {config.batch_config['batch_size']}.")

    # 4. Loop de processamento em lote
    total_extracted_points = 0
    for i in range(0, total_images, config.batch_config['batch_size']):
        batch_ids = image_ids[i:i + config.batch_config['batch_size']]
        batch_num = (i // config.batch_config['batch_size']) + 1
        num_batches = (total_images + config.batch_config['batch_size'] - 1) // config.batch_config['batch_size']

        print(f"\n--- 🔄 Processando Lote {batch_num}/{num_batches} ({len(batch_ids)} imagens) ---")

        # Filtra a coleção para o lote atual de imagens
        batch_collection = ee.ImageCollection(batch_ids)

        # Mapeia a função de extração em toda a coleção do lote (SERVER-SIDE)
        extracted_fc = batch_collection.map(lambda image: extract_data_from_image(image, points_fc)).flatten()

        # Baixa os resultados APENAS para este lote
        try:
            batch_data = extracted_fc.getInfo()['features']
            if not batch_data:
                print("   ⚠️ Nenhum ponto válido extraído neste lote.")
                continue

            # Converte os resultados para um DataFrame
            props_list = [f['properties'] for f in batch_data]
            df_batch = pd.DataFrame(props_list)

            # Adiciona coordenadas a partir da geometria
            geometries = [f['geometry']['coordinates'] for f in batch_data if f.get('geometry')]
            if geometries:
                df_coords = pd.DataFrame(geometries, columns=['longitude', 'latitude'], index=df_batch.index)
                df_batch = pd.concat([df_batch, df_coords], axis=1)

            # Salva o lote no arquivo CSV (anexa se o arquivo já existe)
            is_first_batch = not os.path.exists(output_path)
            df_batch.to_csv(output_path, mode='a', header=is_first_batch, index=False)

            num_extracted = len(df_batch)
            total_extracted_points += num_extracted
            print(f"   ✅ Lote concluído. {num_extracted:,} pontos extraídos e salvos.")

        except Exception as e:
            print(f"   ❌ Erro ao processar ou baixar o lote {batch_num}: {e}")
            print("   Pulando para o próximo lote...")

    # 5. Relatório Final
    end_time = time.time()
    print("\n" + "="*60)
    print("🎉 PIPELINE CONCLUÍDO!")
    print(f"   - Tempo total: {((end_time - start_time) / 60):.2f} minutos")
    print(f"   - Total de pontos extraídos: {total_extracted_points:,}")
    print(f"   - Resultados salvos em: {output_path}")
    print("="*60)


# =============================================================================
# 5. EXECUÇÃO
# =============================================================================

# Para executar, chame a função principal com o objeto de configuração.
# main_batch_pipeline(config)
print("⚡ PRONTO PARA EXECUTAR O SCRIPT OTIMIZADO")
print("Execute: main_batch_pipeline(config)")

🚀 SCRIPT OTIMIZADO: EXTRAÇÃO EM LOTE POR PIXEL
⚡ PRONTO PARA EXECUTAR O SCRIPT OTIMIZADO
Execute: main_batch_pipeline(config)


In [36]:
# -*- coding: utf-8 -*-
"""
SISTEMA DE EXTRAÇÃO DE DADOS EMIT COM ARQUITETURA DE PIPELINE
==================================================================
Versão 3.0 - Refatorado para Programação Orientada a Dados (ETL)

• Estrutura baseada no padrão Extração-Transformação-Carga (ETL).
• Funções com responsabilidades únicas e claras.
• Uso de geradores (generators) para processamento em fluxo e eficiência de memória.
• Configuração centralizada e documentação aprimorada com type hints.

Autor: Sistema Otimizado de Sensoriamento Remoto (Refatorado por Gemini)
Data: Agosto 2025
"""

import ee
import pandas as pd
import os
import time
from datetime import datetime
from typing import List, Dict, Any, Generator, Tuple

# Tenta importar do Colab, se não, ignora.
try:
    from google.colab import drive
except ImportError:
    drive = None

print("🚀 ARQUITETURA DE PIPELINE DE DADOS EMIT (ETL)")
print("="*70)

# =============================================================================
# 1. CONFIGURAÇÃO (O "PROJETO" DO PIPELINE)
# =============================================================================

class PipelineConfig:
    """
    Estrutura de dados que contém todas as configurações do pipeline.
    Funciona como o "contrato" que define o que o pipeline fará.
    """
    def __init__(self):
        self.gee_project = 'ee-vladimircambiental'
        self.paths = {
            'input_points_asset': 'projects/ee-vladimircambiental/assets/pontos_amostrados',
            'study_area_asset': 'projects/ee-crypto/assets/Vegetacao_CE_Shapefile',
            'output_folder': '/content/drive/My Drive/GEE_Data_Pipelines/EMIT_Extraction',
            'output_prefix': 'emit_extraction_results'
        }
        self.temporal = {'start_date': '2022-08-01', 'end_date': datetime.now().strftime('%Y-%m-%d')}
        self.processing = {'batch_size': 20}
        self.selection = {
            'enabled': True,
            'filter_property': 'mapbiomas_class',
            # ⚠️ ATENÇÃO: Use os IDs corretos (como strings) aqui!
            'selected_class_ids': ['3', '4', '5', '49']
        }
        self.schema = {
            'properties_to_keep': ['UniFitoMor', 'mapbiomas_class']
        }

# =============================================================================
# 2. FUNÇÕES DO PIPELINE (LÓGICA ESTRUTURADA)
# =============================================================================

# --- FASE 1: EXTRACT (Extração) ---

def _fetch_gee_data(config: PipelineConfig) -> Tuple[ee.FeatureCollection, ee.ImageCollection]:
    """
    Busca os dados brutos e aplica a amostragem estratificada.
    """
    print("\n--- [E] FASE DE EXTRAÇÃO INICIADA ---")
    try:
        ee.Initialize(project=config.gee_project)
    except Exception:
        ee.Authenticate()
        ee.Initialize(project=config.gee_project)
    print("✅ GEE inicializado.")

    # 1. Carrega todos os pontos do Asset
    print(f"   - Carregando pontos de: {config.paths['input_points_asset']}")
    all_points_fc = ee.FeatureCollection(config.paths['input_points_asset'])

    # 2. Aplica a amostragem estratificada
    if config.selection['enabled']:
        points_fc = _perform_stratified_sample(
            points_fc=all_points_fc,
            class_property=config.selection['filter_property'],
            class_ids=config.selection['selected_class_ids'],
            points_per_class=150  # O número de pontos que você solicitou por classe
        )
    else:
        # Se a seleção não estiver habilitada, usa todos os pontos
        points_fc = all_points_fc

    point_count = points_fc.size().getInfo()
    print(f"   - {point_count:,} pontos selecionados para processamento (APÓS AMOSTRAGEM ESTRATIFICADA).")
    if point_count == 0:
        raise ValueError("A amostragem resultou em 0 pontos. Verifique se as classes existem no Asset.")

    # 3. Carrega a área de estudo e a coleção de imagens (como antes)
    study_area = ee.FeatureCollection(config.paths['study_area_asset']).geometry().dissolve()
    print("   - Carregando coleção de imagens EMIT...")
    image_collection = (ee.ImageCollection('NASA/EMIT/L2A/RFL')
                        .filterDate(config.temporal['start_date'], config.temporal['end_date'])
                        .filterBounds(study_area))
    image_count = image_collection.size().getInfo()
    print(f"   - {image_count} imagens encontradas.")
    if image_count == 0:
        raise ValueError("Nenhuma imagem EMIT encontrada para a área e período.")

    print("--- [E] FASE DE EXTRAÇÃO CONCLUÍDA ---")
    return points_fc, image_collection

# --- FASE 2: TRANSFORM (Transformação) ---
# =============================================================================
# FUNÇÃO DE TRANSFORMAÇÃO CORRIGIDA
# =============================================================================

def _process_batches_generator(points_fc: ee.FeatureCollection, image_collection: ee.ImageCollection, config: PipelineConfig) -> Generator[pd.DataFrame, None, None]:
    """
    Processa os dados em lotes. É um GERADOR: processa um lote, entrega (yields)
    o resultado e libera a memória antes de processar o próximo.
    """
    print("\n--- [T] FASE DE TRANSFORMAÇÃO INICIADA (Processamento em Lote) ---")
    image_ids = image_collection.aggregate_array('system:id').getInfo()
    total_images = len(image_ids)
    batch_size = config.processing['batch_size']
    num_batches = (total_images + batch_size - 1) // batch_size

    # Função server-side para mascarar e extrair dados de uma imagem
    def extract_from_image(image):
        image = ee.Image(image)

        # --- INÍCIO DA CORREÇÃO ---
        # Seleciona a banda de flag e a CONVERTE para inteiro antes das operações de bit
        flag_band = image.select('aggregate_flag').toInt()
        # --- FIM DA CORREÇÃO ---

        # O resto da lógica da máscara agora funciona com a banda inteira
        clean_mask = flag_band.bitwiseAnd(1 << 10).eq(0).And(flag_band.bitwiseAnd(1 << 7).eq(0))

        sampled = image.updateMask(clean_mask).sampleRegions(
            collection=points_fc, scale=60, geometries=True, properties=config.schema['properties_to_keep']
        )
        return sampled.map(lambda f: f.set({'image_id': image.id(), 'image_timestamp': image.get('system:time_start')}))

    for i in range(0, total_images, batch_size):
        batch_ids = image_ids[i:i + batch_size]
        print(f"   - Processando lote {i//batch_size + 1}/{num_batches}...")

        batch_collection = ee.ImageCollection(batch_ids)

        extracted_fc = batch_collection.map(extract_from_image).flatten()

        try:
            batch_data = extracted_fc.getInfo()['features']
            if not batch_data:
                print("     - Nenhum ponto válido extraído neste lote (provavelmente devido à máscara de nuvem).")
                continue

            props_list = [f['properties'] for f in batch_data]
            df_batch = pd.DataFrame(props_list)
            geometries = [f.get('geometry', {}).get('coordinates') for f in batch_data]
            coords = [c for c in geometries if c]
            if coords:
                df_coords = pd.DataFrame(coords, columns=['longitude', 'latitude'], index=df_batch.index)
                df_batch = pd.concat([df_batch, df_coords], axis=1)

            print(f"     - {len(df_batch)} pontos processados.")
            yield df_batch

        except Exception as e:
            # Imprime uma mensagem de erro mais detalhada do GEE
            detailed_error = str(e)
            if "Image.bitwiseAnd" in detailed_error:
                 print(f"     - ❌ Erro de tipo de dado (bitwiseAnd): {detailed_error}. Pulando.")
            else:
                 print(f"     - ❌ Erro ao processar o lote: {detailed_error}. Pulando.")


    print("--- [T] FASE DE TRANSFORMAÇÃO CONCLUÍDA ---")

# --- FASE 3: LOAD (Carga/Salvamento) ---

def _save_results_incrementally(processed_data_generator: Generator[pd.DataFrame, None, None], config: PipelineConfig) -> Tuple[str, int]:
    """
    Recebe o gerador de dados processados e salva cada lote em um arquivo CSV.
    É responsável pela persistência dos dados.
    """
    print("\n--- [L] FASE DE CARGA INICIADA (Salvamento Incremental) ---")
    os.makedirs(config.paths['output_folder'], exist_ok=True)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_path = os.path.join(
        config.paths['output_folder'],
        f"{config.paths['output_prefix']}_{timestamp}.csv"
    )
    print(f"   - Arquivo de saída: {output_path}")

    total_saved_points = 0
    is_first_batch = True
    for batch_df in processed_data_generator:
        batch_df.to_csv(output_path, mode='a', header=is_first_batch, index=False)
        total_saved_points += len(batch_df)
        print(f"   - Lote salvo. Total de pontos no arquivo: {total_saved_points:,}")
        if is_first_batch:
            is_first_batch = False

    print("--- [L] FASE DE CARGA CONCLUÍDA ---")
    return output_path, total_saved_points

# =============================================================================
# 4. ORQUESTRADOR DO PIPELINE
# =============================================================================

def run_pipeline(config: PipelineConfig):
    """
    Orquestra a execução do pipeline ETL.
    É o ponto de entrada principal, com uma lógica limpa e legível.
    """
    start_time = time.time()
    print("\n" + "="*70)
    print("🚀 ORQUESTRANDO EXECUÇÃO DO PIPELINE DE DADOS EMIT")
    print("="*70)

    try:
        # 1. Extrair
        points_fc, image_collection = _fetch_gee_data(config)

        # 2. Transformar (obter o gerador)
        processed_data_generator = _process_batches_generator(points_fc, image_collection, config)

        # 3. Carregar (salvar os resultados do gerador)
        output_file, total_points = _save_results_incrementally(processed_data_generator, config)

        # Relatório Final
        end_time = time.time()
        print("\n" + "="*70)
        print("🎉 PIPELINE EXECUTADO COM SUCESSO!")
        print(f"   - Tempo total: {((end_time - start_time) / 60):.2f} minutos")
        print(f"   - Total de pontos extraídos e salvos: {total_points:,}")
        print(f"   - Resultados em: {output_file}")
        print("="*70)

    except Exception as e:
        print("\n" + "!"*70)
        print(f"💥 ERRO CRÍTICO DURANTE A EXECUÇÃO DO PIPELINE: {e}")
        print("!"*70)

# =============================================================================
# 5. EXECUÇÃO
# =============================================================================

# 1. Crie uma instância da sua configuração
config = PipelineConfig()

# 2. (OPCIONAL) Ajuste qualquer parâmetro antes de executar
# Exemplo:
# config.selection['selected_class_ids'] = ['3'] # Para testar com uma única classe
config.processing['batch_size'] = 10           # Para lotes menores durante o teste

# 3. Execute o pipeline
run_pipeline(config)

🚀 ARQUITETURA DE PIPELINE DE DADOS EMIT (ETL)

🚀 ORQUESTRANDO EXECUÇÃO DO PIPELINE DE DADOS EMIT

--- [E] FASE DE EXTRAÇÃO INICIADA ---
✅ GEE inicializado.
   - Carregando pontos de: projects/ee-vladimircambiental/assets/pontos_amostrados

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
💥 ERRO CRÍTICO DURANTE A EXECUÇÃO DO PIPELINE: name '_perform_stratified_sample' is not defined
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [37]:
def _perform_stratified_sample(
    points_fc: ee.FeatureCollection,
    class_property: str,
    class_ids: List[int],
    points_per_class: int
) -> ee.FeatureCollection:
    """
    Executa uma amostragem estratificada em uma FeatureCollection.

    Args:
        points_fc: A coleção de pontos de entrada.
        class_property: A propriedade usada para definir os estratos (ex: 'mapbiomas_class').
        class_ids: Uma lista de IDs de classe para amostrar.
        points_per_class: O número de pontos a serem amostrados de cada classe.

    Returns:
        Uma FeatureCollection contendo a amostra estratificada.
    """
    print(f"   - Iniciando amostragem estratificada para {len(class_ids)} classes...")

    sampled_collections = []
    for class_id in class_ids:
        # Filtra a coleção para a classe atual
        class_subset = points_fc.filter(ee.Filter.eq(class_property, class_id))

        # Adiciona uma coluna aleatória, ordena e limita para obter a amostra
        sampled_subset = class_subset.randomColumn().sort('random').limit(points_per_class)

        # Armazena a coleção amostrada (um objeto GEE) na lista
        sampled_collections.append(sampled_subset)

    # Une todas as coleções amostradas em uma só
    # ee.FeatureCollection() aceita uma lista de coleções e as une.
    # .flatten() garante que o resultado seja uma única FeatureCollection e não uma coleção de coleções.
    final_sample = ee.FeatureCollection(sampled_collections).flatten()

    return final_sample

In [38]:
def _fetch_gee_data(config: PipelineConfig) -> Tuple[ee.FeatureCollection, ee.ImageCollection]:
    """
    Busca os dados brutos e aplica a amostragem estratificada.
    """
    print("\n--- [E] FASE DE EXTRAÇÃO INICIADA ---")
    try:
        ee.Initialize(project=config.gee_project)
    except Exception:
        ee.Authenticate()
        ee.Initialize(project=config.gee_project)
    print("✅ GEE inicializado.")

    # 1. Carrega todos os pontos do Asset
    print(f"   - Carregando pontos de: {config.paths['input_points_asset']}")
    all_points_fc = ee.FeatureCollection(config.paths['input_points_asset'])

    # 2. Aplica a amostragem estratificada
    if config.selection['enabled']:
        points_fc = _perform_stratified_sample(
            points_fc=all_points_fc,
            class_property=config.selection['filter_property'],
            class_ids=config.selection['selected_class_ids'],
            points_per_class=100  # O número de pontos que você solicitou por classe
        )
    else:
        # Se a seleção não estiver habilitada, usa todos os pontos
        points_fc = all_points_fc

    point_count = points_fc.size().getInfo()
    print(f"   - {point_count:,} pontos selecionados para processamento (APÓS AMOSTRAGEM ESTRATIFICADA).")
    if point_count == 0:
        raise ValueError("A amostragem resultou em 0 pontos. Verifique se as classes existem no Asset.")

    # 3. Carrega a área de estudo e a coleção de imagens (como antes)
    study_area = ee.FeatureCollection(config.paths['study_area_asset']).geometry().dissolve()
    print("   - Carregando coleção de imagens EMIT...")
    image_collection = (ee.ImageCollection('NASA/EMIT/L2A/RFL')
                        .filterDate(config.temporal['start_date'], config.temporal['end_date'])
                        .filterBounds(study_area))
    image_count = image_collection.size().getInfo()
    print(f"   - {image_count} imagens encontradas.")
    if image_count == 0:
        raise ValueError("Nenhuma imagem EMIT encontrada para a área e período.")

    print("--- [E] FASE DE EXTRAÇÃO CONCLUÍDA ---")
    return points_fc, image_collection

In [39]:
# ===================================================================
# PASSO FINAL (CORRIGIDO): CONFIGURAR COM O TIPO DE DADO CORRETO E EXECUTAR
# ===================================================================

# 1. Crie uma instância da configuração
config = PipelineConfig()

# 2. Corrija a lista de classes para usar NÚMEROS INTEIROS, conforme o Asset
config.selection['enabled'] = True
config.selection['selected_class_ids'] = [4] # <-- CORREÇÃO APLICADA AQUI
config.selection['filter_property'] = 'mapbiomas_class'

print(f"✅ Configuração atualizada para usar os IDs numéricos (Integers): {config.selection['selected_class_ids']}")
print("---")

# 3. Execute o pipeline principal com a configuração correta.
run_pipeline(config)

✅ Configuração atualizada para usar os IDs numéricos (Integers): [4]
---

🚀 ORQUESTRANDO EXECUÇÃO DO PIPELINE DE DADOS EMIT

--- [E] FASE DE EXTRAÇÃO INICIADA ---
✅ GEE inicializado.
   - Carregando pontos de: projects/ee-vladimircambiental/assets/pontos_amostrados
   - Iniciando amostragem estratificada para 1 classes...
   - 100 pontos selecionados para processamento (APÓS AMOSTRAGEM ESTRATIFICADA).
   - Carregando coleção de imagens EMIT...


KeyboardInterrupt: 

# **ETL 3.0**:



---

ESTRATIFICAÇÃO MULTIVARIADA

In [47]:
# -*- coding: utf-8 -*-
"""
SISTEMA DE EXTRAÇÃO DE DADOS EMIT COM ARQUITETURA DE PIPELINE
==================================================================
Versão 3.0 - Refatorado para Programação Orientada a Dados (ETL)

• Estrutura baseada no padrão Extração-Transformação-Carga (ETL).
• Funções com responsabilidades únicas e claras.
• Uso de geradores (generators) para processamento em fluxo e eficiência de memória.
• Configuração centralizada e documentação aprimorada com type hints.

Autor: Sistema Otimizado de Sensoriamento Remoto (Refatorado por Gemini)
Data: Agosto 2025
"""

import ee
import pandas as pd
import os
import time
from datetime import datetime
from typing import List, Dict, Any, Generator, Tuple

# Tenta importar do Colab, se não, ignora.
try:
    from google.colab import drive
except ImportError:
    drive = None

print("🚀 ARQUITETURA DE PIPELINE DE DADOS EMIT (ETL)")
print("="*70)

# =============================================================================
# NOVA FUNÇÃO AUXILIAR: AMOSTRAGEM ESTRATIFICADA SIMPLES
# =============================================================================
def _perform_single_stratified_sample(
    points_fc: ee.FeatureCollection,
    strata_property: str,
    points_per_stratum: int
) -> ee.FeatureCollection:
    """
    Executa amostragem estratificada em uma única propriedade.

    Args:
        points_fc: A coleção de feições a ser amostrada.
        strata_property: A propriedade usada para definir os estratos (ex: 'mapbiomas_class').
        points_per_stratum: O número de pontos a ser em sorteados de cada estrato.

    Returns:
        Uma FeatureCollection contendo os pontos amostrados.
    """
    print(f"   - Iniciando amostragem estratificada simples por '{strata_property}'...")

    # 1. Obtém a lista de estratos únicos (classes) do GEE
    unique_strata = points_fc.aggregate_array(strata_property).distinct().getInfo()
    print(f"   - Encontrados {len(unique_strata)} estratos de '{strata_property}': {unique_strata}")

    if not unique_strata:
        return ee.FeatureCollection([])

    # 2. Itera sobre cada estrato, filtra, amostra e coleta os resultados
    sampled_collections = []
    for stratum in unique_strata:
        # Filtra a coleção para incluir apenas os pontos do estrato atual
        stratum_subset = points_fc.filter(ee.Filter.eq(strata_property, stratum))

        # Realiza a amostragem aleatória dentro do estrato
        # .randomColumn() adiciona uma coluna 'random' com valores de 0 a 1
        # .sort('random') embaralha os pontos
        # .limit() pega o número desejado de pontos
        sampled_subset = stratum_subset.randomColumn().sort('random').limit(points_per_stratum)

        sampled_collections.append(sampled_subset)

    # 3. Une todas as coleções amostradas em uma só
    final_sample = ee.FeatureCollection(sampled_collections).flatten()

    return final_sample


# =============================================================================
# 1. CONFIGURAÇÃO (ATUALIZADA)
# =============================================================================
class PipelineConfig:
    """
    Estrutura de dados que contém todas as configurações do pipeline.
    """
    def __init__(self):
        self.gee_project = 'ee-vladimircambiental'
        self.paths = {
            'input_points_asset': 'projects/ee-vladimircambiental/assets/pontos_amostrados',
            'study_area_asset': 'projects/ee-crypto/assets/APA_chapada_araripe',
            'output_folder': '/content/drive/My Drive/GEE_Data_Pipelines/EMIT_Preconfigured_Filter',
            'output_prefix': 'emit_extract'
        }
        self.temporal = {'start_date': '2022-08-01', 'end_date': datetime.now().strftime('%Y-%m-%d')}
        self.processing = {'batch_size': 20}
        self.schema = {
            'properties_to_keep': ['UniFitoMor', 'mapbiomas_class']
        }

        # Modo Interativo (desabilitado para este caso)
        self.interactive_mode = {
            'enabled': False,
            'filter_column': 'UniFitoMor',
            'stratify_column': 'mapbiomas_class',
            'points_per_stratum': 15
        }

        # --- NOVA SEÇÃO PARA FILTRO PRÉ-CONFIGURADO ---
        # Usado quando o modo interativo está desabilitado.
        self.preconfigured_filter = {
            'enabled': True,  # Ativa o filtro estático
            'filter_column': 'UniFitoMor', # A coluna pela qual filtrar
            'selected_values': ['Carrasco'] # O(s) valor(es) a ser(em) extraído(s). Use strings!
        }

        # --- Adicionado a seção 'selection' que estava faltando ---
        self.selection = {
            'enabled': True,
            'filter_property_1': 'UniFitoMor', # Primeira propriedade para estratificação
            'filter_property_2': 'mapbiomas_class', # Segunda propriedade para estratificação
            'points_per_class': 10 # Número de pontos por estrato combinado (UniFitoMor + mapbiomas_class)
        }
# =============================================================================
# 5. EXECUÇÃO (ATUALIZADA)
# =============================================================================

# 1. Crie uma instância da sua configuração
config = PipelineConfig()

# 2. (OPCIONAL) Ajuste qualquer parâmetro antes de executar
config.selection['points_per_class'] = 1 # Ex: Reduzir pontos por estrato para um teste rápido
config.processing['batch_size'] = 1

print("✅ Configuração de amostragem estratificada dupla pronta.")
print(f"   - Estratificando por: '{config.selection['filter_property_1']}' e '{config.selection['filter_property_2']}'")
print(f"   - Pontos por estrato combinado: {config.selection['points_per_class']}")
print("---")


# 3. Execute o pipeline com a nova configuração
run_pipeline(config)
# =============================================================================
# 2. FUNÇÕES DO PIPELINE (LÓGICA ESTRUTURADA)
# =============================================================================

# --- FASE 1: EXTRACT (Extração) ---
# SUBSTITUA ESTA FUNÇÃO
def _fetch_gee_data(config: PipelineConfig) -> Tuple[ee.FeatureCollection, ee.ImageCollection]:
    """
    Busca os dados brutos e aplica a amostragem estratificada dupla sem usar .getInfo() para contar.
    """
    print("\\n--- [E] FASE DE EXTRAÇÃO INICIADA ---")
    try:
        ee.Initialize(project=config.gee_project)
    except Exception:
        ee.Authenticate()
        ee.Initialize(project=config.gee_project)
    print("✅ GEE inicializado.")

    print(f"   - Carregando pontos de: {config.paths['input_points_asset']}")
    all_points_fc = ee.FeatureCollection(config.paths['input_points_asset'])

    if config.selection['enabled']:
        # --- INÍCIO DA MUDANÇA ---
        points_fc, unique_strata_list = _perform_double_stratified_sample(
            points_fc=all_points_fc,
            property1=config.selection['filter_property_1'],
            property2=config.selection['filter_property_2'],
            points_per_stratum=config.selection['points_per_class']
        )

        # A nova verificação é feita na lista de estratos, que já está na memória local
        if not unique_strata_list:
            raise ValueError("A amostragem resultou em 0 estratos. Verifique as propriedades no Asset.")

        # Imprime uma contagem teórica, sem forçar computação no GEE
        theoretical_max = len(unique_strata_list) * config.selection['points_per_class']
        print(f"   - Amostragem definida. Máximo teórico de {theoretical_max:,} pontos a serem processados.")
        # --- FIM DA MUDANÇA ---
    else:
        points_fc = all_points_fc

    # O resto da função permanece igual
    study_area = ee.FeatureCollection(config.paths['study_area_asset']).geometry().dissolve()
    print("   - Carregando coleção de imagens EMIT...")
    image_collection = (ee.ImageCollection('NASA/EMIT/L2A/RFL')
                        .filterDate(config.temporal['start_date'], config.temporal['end_date'])
                        .filterBounds(study_area))
    image_count = image_collection.size().getInfo()
    print(f"   - {image_count} imagens encontradas.")
    if image_count == 0:
        raise ValueError("Nenhuma imagem EMIT encontrada para a área e período.")

    print("--- [E] FASE DE EXTRAÇÃO CONCLUÍDA ---")
    return points_fc, image_collection

# --- FASE 2: TRANSFORM (Transformação) ---
# =============================================================================
# FUNÇÃO DE TRANSFORMAÇÃO CORRIGIDA
# =============================================================================

def _process_batches_generator(points_fc: ee.FeatureCollection, image_collection: ee.ImageCollection, config: PipelineConfig) -> Generator[pd.DataFrame, None, None]:
    """
    Processa os dados em lotes. É um GERADOR: processa um lote, entrega (yields)
    o resultado e libera a memória antes de processar o próximo.
    """
    print("\n--- [T] FASE DE TRANSFORMAÇÃO INICIADA (Processamento em Lote) ---")
    image_ids = image_collection.aggregate_array('system:id').getInfo()
    total_images = len(image_ids)
    batch_size = config.processing['batch_size']
    num_batches = (total_images + batch_size - 1) // batch_size

    # Função server-side para mascarar e extrair dados de uma imagem
    def extract_from_image(image):
        image = ee.Image(image)

        # --- INÍCIO DA CORREÇÃO ---
        # Seleciona a banda de flag e a CONVERTE para inteiro antes das operações de bit
        flag_band = image.select('aggregate_flag').toInt()
        # --- FIM DA CORREÇÃO ---

        # O resto da lógica da máscara agora funciona com a banda inteira
        clean_mask = flag_band.bitwiseAnd(1 << 10).eq(0).And(flag_band.bitwiseAnd(1 << 7).eq(0))

        sampled = image.updateMask(clean_mask).sampleRegions(
            collection=points_fc, scale=60, geometries=True, properties=config.schema['properties_to_keep']
        )
        return sampled.map(lambda f: f.set({'image_id': image.id(), 'image_timestamp': image.get('system:time_start')}))

    for i in range(0, total_images, batch_size):
        batch_ids = image_ids[i:i + batch_size]
        print(f"   - Processando lote {i//batch_size + 1}/{num_batches}...")

        batch_collection = ee.ImageCollection(batch_ids)

        extracted_fc = batch_collection.map(extract_from_image).flatten()

        try:
            batch_data = extracted_fc.getInfo()['features']
            if not batch_data:
                print("     - Nenhum ponto válido extraído neste lote (provavelmente devido à máscara de nuvem).")
                continue

            props_list = [f['properties'] for f in batch_data]
            df_batch = pd.DataFrame(props_list)
            geometries = [f.get('geometry', {}).get('coordinates') for f in batch_data]
            coords = [c for c in geometries if c]
            if coords:
                df_coords = pd.DataFrame(coords, columns=['longitude', 'latitude'], index=df_batch.index)
                df_batch = pd.concat([df_batch, df_coords], axis=1)

            print(f"     - {len(df_batch)} pontos processados.")
            yield df_batch

        except Exception as e:
            # Imprime uma mensagem de erro mais detalhada do GEE
            detailed_error = str(e)
            if "Image.bitwiseAnd" in detailed_error:
                 print(f"     - ❌ Erro de tipo de dado (bitwiseAnd): {detailed_error}. Pulando.")
            else:
                 print(f"     - ❌ Erro ao processar o lote: {detailed_error}. Pulando.")


    print("--- [T] FASE DE TRANSFORMAÇÃO CONCLUÍDA ---")

# --- FASE 3: LOAD (Carga/Salvamento) ---

def _save_results_incrementally(processed_data_generator: Generator[pd.DataFrame, None, None], config: PipelineConfig) -> Tuple[str, int]:
    """
    Recebe o gerador de dados processados e salva cada lote em um arquivo CSV.
    É responsável pela persistência dos dados.
    """
    print("\n--- [L] FASE DE CARGA INICIADA (Salvamento Incremental) ---")
    os.makedirs(config.paths['output_folder'], exist_ok=True)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_path = os.path.join(
        config.paths['output_folder'],
        f"{config.paths['output_prefix']}_{timestamp}.csv"
    )
    print(f"   - Arquivo de saída: {output_path}")

    total_saved_points = 0
    is_first_batch = True
    for batch_df in processed_data_generator:
        batch_df.to_csv(output_path, mode='a', header=is_first_batch, index=False)
        total_saved_points += len(batch_df)
        print(f"   - Lote salvo. Total de pontos no arquivo: {total_saved_points:,}")
        if is_first_batch:
            is_first_batch = False

    print("--- [L] FASE DE CARGA CONCLUÍDA ---")
    return output_path, total_saved_points

# =============================================================================
# 4. ORQUESTRADOR DO PIPELINE
# =============================================================================

def run_pipeline(config: PipelineConfig):
    """
    Orquestra a execução do pipeline ETL.
    É o ponto de entrada principal, com uma lógica limpa e legível.
    """
    start_time = time.time()
    print("\n" + "="*70)
    print("🚀 ORQUESTRANDO EXECUÇÃO DO PIPELINE DE DADOS EMIT")
    print("="*70)

    try:
        # 1. Extrair
        points_fc, image_collection = _fetch_gee_data(config)

        # 2. Transformar (obter o gerador)
        processed_data_generator = _process_batches_generator(points_fc, image_collection, config)

        # 3. Carregar (salvar os resultados do gerador)
        output_file, total_points = _save_results_incrementally(processed_data_generator, config)

        # Relatório Final
        end_time = time.time()
        print("\n" + "="*70)
        print("🎉 PIPELINE EXECUTADO COM SUCESSO!")
        print(f"   - Tempo total: {((end_time - start_time) / 60):.2f} minutos")
        print(f"   - Total de pontos extraídos e salvos: {total_points:,}")
        print(f"   - Resultados em: {output_file}")
        print("="*70)

    except Exception as e:
        print("\n" + "!"*70)
        print(f"💥 ERRO CRÍTICO DURANTE A EXECUÇÃO DO PIPELINE: {e}")
        print("!"*70)



# 3. Execute o pipeline
run_pipeline(config)

🚀 ARQUITETURA DE PIPELINE DE DADOS EMIT (ETL)
✅ Configuração de amostragem estratificada dupla pronta.
   - Estratificando por: 'UniFitoMor' e 'mapbiomas_class'
   - Pontos por estrato combinado: 1
---

🚀 ORQUESTRANDO EXECUÇÃO DO PIPELINE DE DADOS EMIT
\n--- [E] FASE DE EXTRAÇÃO INICIADA ---
✅ GEE inicializado.
   - Carregando pontos de: projects/ee-vladimircambiental/assets/pontos_amostrados
   - Iniciando amostragem estratificada dupla por 'UniFitoMor' e 'mapbiomas_class'...
   - Encontrados 119 estratos únicos combinados.
   - Amostragem definida. Máximo teórico de 119 pontos a serem processados.
   - Carregando coleção de imagens EMIT...
   - 21 imagens encontradas.
--- [E] FASE DE EXTRAÇÃO CONCLUÍDA ---

--- [L] FASE DE CARGA INICIADA (Salvamento Incremental) ---
   - Arquivo de saída: /content/drive/My Drive/GEE_Data_Pipelines/EMIT_Preconfigured_Filter/emit_extract_20250901_192931.csv

--- [T] FASE DE TRANSFORMAÇÃO INICIADA (Processamento em Lote) ---
   - Processando lote 1/21..

     - ❌ Erro ao processar o lote: Computation timed out.. Pulando.
   - Processando lote 2/21...


KeyboardInterrupt: 